In [1]:
import sys
sys.path.append('../')
import requests
from atlassian import Confluence
import time
from getpass import getpass
from Comala.Comala_Util import Comala_Util
from Confluence.Confluence_Util import Confluence_Util
from ConfiForm.ConfiForm_Util import ConfiForm_Utils



contentApiUrl = '/rest/api/content'
# Change these based on your instance
confluenceBaseUrl = "https://confluence.corp.bioagilytix.com/confluence"


username = 'yangyang.liu'
password = getpass("password")


confluenceUtil = Confluence_Util(confluenceBaseUrl, username, password)
comalaUtil = Comala_Util(confluenceBaseUrl, username, password)
confiFromUtil = ConfiForm_Utils(confluenceBaseUrl, username, password)
confluence = Confluence(
    url='https://confluence.corp.bioagilytix.com/confluence',
    username=username,
    password=password)


In [2]:

allSpace = set()
for space in confluence.get_all_spaces(start=0, limit=2000):
    allSpace.add(space['key'])
for space in confluence.get_all_spaces(start=500, limit=2000):
    allSpace.add(space['key'])


In [3]:
#print(comalaUtil.isInitial(28607721))
pageIdDict = dict()
for space in allSpace:
    pageIdList = list()
    for pageId in confluenceUtil.get_all_page_id_from_space(space):
        pageIdList.append(pageId)
    pageIdDict[space] = pageIdList


Comala_Status_Dictionary = dict()
for spaceKey in pageIdDict.keys():
    status_dict = dict()
    for pageId in pageIdDict[spaceKey]:
        status = comalaUtil.getState(pageId)
        status_dict[pageId] = status
        time.sleep(1)
    Comala_Status_Dictionary[spaceKey] = status_dict


https://confluence.corp.bioagilytix.com/confluence/rest/api/space/BAL21099151/content?start=0&limit=9999&type=page
<Response [200]>
https://confluence.corp.bioagilytix.com/confluence/rest/api/space/BAL21620034/content?start=0&limit=9999&type=page
<Response [200]>
https://confluence.corp.bioagilytix.com/confluence/rest/api/space/BAL21311326/content?start=0&limit=9999&type=page
<Response [200]>
https://confluence.corp.bioagilytix.com/confluence/rest/api/space/BAL21143143/content?start=0&limit=9999&type=page
<Response [200]>
https://confluence.corp.bioagilytix.com/confluence/rest/api/space/BAL21602037/content?start=0&limit=9999&type=page
<Response [200]>
https://confluence.corp.bioagilytix.com/confluence/rest/api/space/BAL21285199/content?start=0&limit=9999&type=page
<Response [200]>
https://confluence.corp.bioagilytix.com/confluence/rest/api/space/BAL21713041/content?start=0&limit=9999&type=page
<Response [200]>
https://confluence.corp.bioagilytix.com/confluence/rest/api/space/LABRepo/co

AttributeError: 'Comala_Util' object has no attribute 'getState'

In [3]:
##print(ConfiForm_Utils.getConfiFormEntries(28607721, 'runinfo'))


TypeError: getConfiFormEntries() missing 1 required positional argument: 'formName'

In [11]:
def getConfiFormEntries(pageId, formName):
    requestUrl = "https://confluence.corp.bioagilytix.com/confluence/ajax/confiforms/rest/filter.action?pageId={}&f={}".format(pageId, formName)
    requestResponse = requests.get(requestUrl, auth=(username, password))
    
    return requestResponse.json()['list']['entry']
    


In [12]:
print(getConfiFormEntries(28607721, 'runinfo'))


[{'recordId': 2, 'createdBy': 'yangyang.liu', 'created': 1641886287889, 'id': 'a178b2cc-71c4-4955-b0e1-e2ded31a69a0', 'fields': {'submitted': True, 'rundesc': 'UMR 106 Cell Preparation for Assay_PTHrP_Method_Worksheet', 'flaskBatchID': 2, 'flaskNumbers': 2, 'runid': 2, 'startdate': 1641859200000, 'class': 'fields', 'worksheetDate': 1642550400000, 'reasonForChange': 'Original Entry'}, 'ownedBy': 'yangyang.liu'}]


In [7]:
def getPageBody(pageId):
    requestUrl = "{}/rest/api/content/{}?expand=body.storage".format(
        confluenceBaseUrl, pageId)
    requestResponse = requests.get(requestUrl, auth=(username, password))
    return requestResponse.json()['body']['storage']['value']

In [8]:
bodyStorage = getPageBody(28607721)


In [35]:
from bs4 import BeautifulSoup


def getFormNames(bodyStorage):
    parsed_html = BeautifulSoup(bodyStorage)
    form = parsed_html.find('ac:structured-macro', attrs={"ac:name": "confiform"})
    print("{} Forms Found on the page".format(len(form.findAll('ac:parameter', attrs={"ac:name": "formName"}))))
    formList = set()
    for formName in form.findAll('ac:parameter', attrs={"ac:name": "formName"}):
       
        formList.add(formName.text)
    return formList


In [36]:
print(getFormNames(bodyStorage))

3 Forms Found on the page
{'runinfo'}


In [4]:
Comala_Status_Dictionary

{'BAL21099151': {'28638711': 'CW Not Found',
  '28638705': 'CW Not Found',
  '28638723': 'Draft',
  '28638736': 'CW Not Found',
  '28638737': 'Draft',
  '28638731': 'CW Not Found',
  '28638703': 'Draft',
  '28638710': 'Draft',
  '28638702': 'CW Not Found',
  '28638716': 'Draft',
  '28638713': 'Draft',
  '28638717': 'CW Not Found',
  '28638735': 'CW Not Found',
  '28638732': 'Draft',
  '28638733': 'Draft',
  '28638734': 'Draft',
  '28638715': 'Draft',
  '28638709': 'CW Not Found',
  '28638712': 'Draft',
  '28638727': 'CW Not Found',
  '28638719': 'Draft',
  '28638729': 'CW Not Found',
  '28638730': 'Analyst',
  '28638721': 'Draft',
  '28638738': 'CW Not Found',
  '28638707': 'Draft',
  '28638718': 'Draft',
  '28638704': 'Draft',
  '28638726': 'Draft',
  '28638714': 'Draft',
  '28638722': 'Draft',
  '28638724': 'Draft',
  '28638725': 'Draft',
  '28638728': 'Draft',
  '28638720': 'Draft'},
 'BAL21620034': {'28619218': 'CW Not Found',
  '28619212': 'CW Not Found',
  '28619229': 'Draft',
  

In [5]:
import pickle


In [6]:
with open('Comala_Status_All_Page.json', 'wb') as fp:
    pickle.dump(Comala_Status_Dictionary, fp)
